# Import de news dans Elasticsearch
Le jeu de données provient de https://www.kaggle.com/datasets/rmisra/news-category-dataset?resource=download.
Il contient `210,294` news entre 2012 et 2022. Le format des news est le suivant : 
```
{
  "link": "https://www.huffpost.com/entry/covid-boosters-uptake-us_n_632d719ee4b087fae6feaac9",
  "headline": "Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID Boosters",
  "category": "U.S. NEWS",
  "short_description": "Health experts said it is too early to predict whether demand would match up with the 171 million doses of the new boosters the U.S. ordered for the fall.",
  "authors": "Carla K. Johnson, AP",
  "date": "2022-09-23"
}
```

L'objectif de ce notebook est d'importer ces données dans Elasticsearch afin de réaliser des recherches sémantiques basées sur l'utilisation de modèles de langage type BERT.

In [ ]:
# Configuration
import json # Pour lire le fichier JSON
from opensearchpy import OpenSearch # API Python pour OpenSearch

OS_HOST='localhost'
OS_PORT=9200
NEWS_JSON_PATH='../data/News_Category_Dataset_v3.json'
INDEX_NAME='news'
MAX_NEWS=10000 # Nombre maximum d'articles à indexer

In [ ]:
# Ingestion des données
def payload_constructor(data):
    '''Coonstruction du payload pour l'indexation en masse'''
    payload_string = ''
    for datum in data:
        action = {'index': {'_id': datum['link']}}
        action_string = json.dumps(action) + '\n'
        payload_string += action_string
        this_line = json.dumps(datum) + '\n'
        payload_string += this_line
    return payload_string


index_name = INDEX_NAME
batch_size = 1000 # Nombre d'articles à indexer à chaque fois

client = OpenSearch(
    hosts=[{'host': 'localhost', 'port': 9200}],
    http_compress=True,
)

with open(NEWS_JSON_PATH) as f:
    lines = f.readlines()

processed_lines = 0
for start in range(0, len(lines), batch_size):
    data = []
    for line in lines[start:start+batch_size]:
        data.append(json.loads(line))
        processed_lines += 1
    response = client.bulk(body=payload_constructor(data), index=index_name)
    print("{} lines processed.".format(processed_lines))
    if processed_lines >= MAX_NEWS:
        print('{} lines reached. Abort indexing'.format(MAX_NEWS))
        break